Specifications - Financial Data
Your stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)

As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

Adding extra years as a requiremet for part4

In [1]:
import os, time,json
import tmdbsimple as tmdb 
import pandas as pd
from tqdm.notebook import tqdm_notebook
import time

In [2]:
#Connect

In [3]:
with open('/Users/siblose/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [4]:
tmdb.API_KEY =  login['api-key']
FOLDER = "Project3Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2002.csv.gz',
 'final_tmdb_data_2003.csv.gz',
 'final_tmdb_data_2004.csv.gz',
 'final_tmdb_data_2005.csv.gz',
 'final_tmdb_data_2006.csv.gz',
 'final_tmdb_data_2007.csv.gz',
 'final_tmdb_data_2008.csv.gz',
 'final_tmdb_data_2009.csv.gz',
 'final_tmdb_data_2010.csv.gz',
 'final_tmdb_data_2011.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'final_tmdb_data_2013.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'final_tmdb_data_2015.csv.gz',
 'final_tmdb_data_2016.csv.gz',
 'final_tmdb_data_2019.csv.gz',
 'final_tmdb_data_2020.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json',
 'tmdb_api_results_2003.json',
 'tmdb_api_results_2004.json',
 'tmdb_api_results_2005.json',
 'tmdb_api_results_2006.json',
 'tmdb_api_results_2007.json',
 'tmdb_api_results_2008.json',
 'tmdb_api_results_2009.json',
 'tmdb_api_

In [5]:
#function will help you append/extend a JSON file with Python
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [6]:
#It should return a dictionary of results that includes certification.
def get_movie_with_rating(movie_id):
  #Get movie object for the current id
  movie = tmdb.Movies(movie_id)    
  # Save the .info .release dictionaries
  info = movie.info()
   
  releases = movie.releases()  
  #Loop through countries in releases
  for c in releases['countries']:
     # if the country abbreviation==US
    if c['iso_3166_1'] == 'US':
          ## save a "certification" key in the info dict with the certification
       info['certification'] = c['certification']
       break
  return info

In [7]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Project3Data/title_basics.csv.gz')

In [8]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
3,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
4,tt0001285,movie,The Life of Moses,The Life of Moses,0,1909,NaN,50,"Biography,Drama,Family"


In [9]:
#As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001
YEARS_TO_GET = [2017,2018,2019,2020]

In [10]:
#Define an errors list
errors = []

In [11]:
#Each year should be saved as a separate .csv.gz file

In [12]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
      with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    
      #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)
print(f"- Total errors: {len(errors)}")

YEARS:   0%|          | 0/4 [00:00<?, ?it/s]

Movies from 2017:   0%|          | 0/2988 [00:00<?, ?it/s]

Movies from 2018:   0%|          | 0/5802 [00:00<?, ?it/s]

Movies from 2019:   0%|          | 0/4478 [00:00<?, ?it/s]

Movies from 2020:   0%|          | 0/5031 [00:00<?, ?it/s]

- Total errors: 6759


In [13]:
final_year_df.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0062336,0.0,/fw5tsNib4QZBEw18xmebpVe3WZ8.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",http://poetastros.com/el-tango-del-viudo/,602986.0,es,El tango del viudo y su espejo deformante,...,0.0,63.0,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,,The Tango of the Widower and Its Distorting Mi...,0.0,6.167,6.0,NaN
2,tt0805647,0.0,/8rIoyM6zYXJNjzGseT3MRusMPWl.jpg,None,0.0,"[{'id': 10751, 'name': 'Family'}, {'id': 35, '...",https://www.thewitchesmovie.net/,531219.0,en,Roald Dahl's The Witches,...,29303571.0,106.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They're real!,Roald Dahl's The Witches,0.0,6.391,2609.0,PG
3,tt0920462,0.0,/7GZ60FQuzejs73tfoBDFpCR21Iv.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}]",https://www.lostgirlslovehotels.film,479259.0,en,Lost Girls & Love Hotels,...,106045.0,97.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Lost Girls & Love Hotels,0.0,5.360,157.0,R
4,tt0926132,0.0,/op14w44FvLUqH7TWyT1ijzrSfXV.jpg,None,0.0,"[{'id': 27, 'name': 'Horror'}]",http://www.roadtored.com,651448.0,en,Darkslide,...,0.0,121.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Keep Moving or Die,Darkslide,0.0,5.200,5.0,


Confirm Your API Function works.

In [14]:
 #test your function on these 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook").

In [15]:
test = get_movie_with_rating("tt0848228") #put your function name here
test

ConnectionError: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/movie/tt0848228?api_key=ee7d38b3e9654c4fdc9c2c73123594f0 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000154314B6D40>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
test1 = get_movie_with_rating("tt0332280") #put your function name here
test1

In [ ]:
#Exploratory Data Analysis in file 2

